In [4]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

# bias correction

In [7]:
ds_bias = xr.open_dataset('bias_era5_cmip6_1120.nc')
bias_cmip6_era5 = ds_bias['bias_era5_cmip6_1120']

In [8]:
#take five years in SSP1-2.6 for example
ds_cmip6_4044 = xr.open_dataset('t2_4044_ssp126_lu.nc')
t2_4044_cmip6_original = ds_cmip6_4044['t2_4044_ssp126_lu']

In [9]:
t2_4044_cmip6_df = t2_4044_cmip6_original.to_dataframe().reset_index()
bias_cmip6_era5_df = bias_cmip6_era5.to_dataframe().reset_index()

In [10]:
t2_4044_cmip6_df['month'] = t2_4044_cmip6_df.Time.dt.month
t2_4044_cmip6_df['hour'] = t2_4044_cmip6_df.Time.dt.hour

In [11]:
del bias_cmip6_era5_df['XLAT']
del bias_cmip6_era5_df['XLONG']

In [12]:
t2_4044_cmip6_df = pd.merge(t2_4044_cmip6_df,bias_cmip6_era5_df,how='outer',on=['month','hour','south_north','west_east'])

In [15]:
t2_4044_cmip6_df['t2_4044_ssp126_lu'] = t2_4044_cmip6_df['t2_4044_ssp126_lu']+t2_4044_cmip6_df['bias_era5_cmip6_1120']

In [19]:
del t2_4044_cmip6_df['month']
del t2_4044_cmip6_df['hour']
del t2_4044_cmip6_df['bias_era5_cmip6_1120']
del t2_4044_cmip6_df['XLAT']
del t2_4044_cmip6_df['XLONG']

In [20]:
t2_4044_cmip6_df.sort_values(['Time','south_north','west_east'],ascending=[True,True,True],inplace=True)

In [22]:
t2_4044_cmip6 = t2_4044_cmip6_df.t2_4044_ssp126_lu.values 
t2_4044_cmip6 = t2_4044_cmip6.reshape(14640,162,213)

In [23]:
Time = t2_4044_cmip6_original.Time
XLAT = t2_4044_cmip6_original.XLAT
XLONG = t2_4044_cmip6_original.XLONG
south_north = t2_4044_cmip6_original.south_north
west_east = t2_4044_cmip6_original.west_east

In [24]:
#extract the land use data from one of the WRF output files
ds_lu_1 = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu = ds_lu_1['LU_INDEX']

lu = lu[0,:,:]

del lu.coords['XTIME']

In [25]:
t2_4044_era5_corrected = xr.DataArray(t2_4044_cmip6,
                                       dims=["Time","south_north","west_east"],
                                       coords=dict(
                                       XLONG=(["south_north", "west_east"], XLONG),
                                       XLAT=(["south_north", "west_east"], XLAT),
                                       Time=Time,
                                       LU_INDEX=lu))

In [26]:
ds = xr.Dataset({'t2_4044_era5_corrected': t2_4044_era5_corrected})
ds.to_netcdf('t2_4044_era_corrected.nc')